
---

# Transformaciones
---


In [5]:
import pandas as pd
import ast
df = pd.read_csv('Movies\\movies_dataset.csv', low_memory=False)

In [2]:

'''Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.'''
df = df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])


In [3]:
'''Los valores nulos del campo release date deben eliminarse.'''

df = df.dropna(subset=['release_date'])

In [15]:
'''Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.'''

df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)

In [16]:
'''De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.'''

#convertimos las fechas al datatype datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') 
df['release_year'] = df['release_date'].dt.year

In [17]:
'''Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget,
cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.'''
# Convertir las columnas 'revenue' y 'budget' a tipo numérico
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
# Rellenar valores nulos en 'revenue' y 'budget' con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

# Crear la columna 'return' calculando revenue / budget
# Usar np.where para evitar división por 0
import numpy as np
df['return'] = np.where(df['budget'] == 0, 0, df['revenue'] / df['budget'])

In [18]:
df['return'].head(5)

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

In [19]:
''' Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una
lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API!
O bien buscar la manera de acceder a esos datos sin desanidarlos.'''
# Función para convertir cadenas a diccionarios o listas
def convertir_en_obj(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return x  # Retorna la cadena si no es un diccionario válido
    return x

# Leer el archivo CSV
df = pd.read_csv('C:/Users/fede/Desktop/movies_dataset.csv')

# Aplicar la conversión a las series 
df['belongs_to_collection'] = df['belongs_to_collection'].apply(convertir_en_obj)
df['genres'] = df['genres'].apply(convertir_en_obj)
df['production_companies'] = df['production_companies'].apply(convertir_en_obj)
df['production_countries'] = df['production_countries'].apply(convertir_en_obj)
df['spoken_languages'] = df['spoken_languages'].apply(convertir_en_obj)

# Desanidar diccionarios
BTC_df = df['belongs_to_collection'].apply(pd.Series)
BTC_df = BTC_df.drop(columns = 0, errors='ignore')  # Eliminar columna de índice si existe
nuevos_nombres_BTC = ['BTCid', 'BTCname', 'BTCposter_path', 'BTCbackdrop_path']
BTC_df.columns = nuevos_nombres_BTC

# Desanidar listas
genres_df = df['genres'].apply(pd.Series)
production_companies_df = df['production_companies'].apply(pd.Series)
production_countries_df = df['production_countries'].apply(pd.Series)
spoken_languages_df = df['spoken_languages'].apply(pd.Series)

# Asegurarse de que no haya nombres de columnas duplicados
genres_df = genres_df.rename(columns=lambda x: f'genre_{x}')
production_companies_df = production_companies_df.rename(columns=lambda x: f'company_{x}')
production_countries_df = production_countries_df.rename(columns=lambda x: f'country_{x}')
spoken_languages_df = spoken_languages_df.rename(columns=lambda x: f'language_{x}')

# Eliminar las columnas originales
df = df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'])

# Unir los DataFrames desanidado al DataFrame original
df = df.join(BTC_df)
df = df.join(genres_df)
df = df.join(production_companies_df)
df = df.join(production_countries_df)
df = df.join(spoken_languages_df)


C:\Users\fede\AppData\Local\Temp\ipykernel_2252\3678100755.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/fede/Desktop/movies_dataset.csv')
C:\Users\fede\AppData\Local\Temp\ipykernel_2252\3678100755.py:24: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  BTC_df = df['belongs_to_collection'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_2252\3678100755.py:30: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  genres_df = df['genres'].apply(pd.Series)
C:\Users\fede\AppData\Local\Temp\ipykernel_2252\3678100755.py:31: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  producti

In [20]:
df.head(3)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,language_9,language_10,language_11,language_12,language_13,language_14,language_15,language_16,language_17,language_18
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
